In [1]:
import pandas as pd
import time
import random

from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

bebidas = {
    "Bebidas refrescantes": "https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=",
    "Cervezas":"https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page=",
    "Vino Tinto":"https://www.alcampo.es/compra-online/bebidas/vino-tinto/c/W1151?q=%3Arelevance&page=",
    # "Zumos de Frutas":"https://www.alcampo.es/compra-online/bebidas/zumos-de-frutas/c/W1102?q=%3Arelevance&page=",
    # "Bebidas Alcohólicas":"https://www.alcampo.es/compra-online/bebidas/bebidas-alcoholicas/c/W1154?q=%3Arelevance&page=",
    # "Vino Blanco": "https://www.alcampo.es/compra-online/bebidas/vino-blanco/c/W1152?q=%3Arelevance&page=",
    # "Agua, Soda y Gaseosas": "https://www.alcampo.es/compra-online/bebidas/agua-soda-y-gaseosas/c/W1101?q=%3Arelevance&page=",
    # "Licores": "https://www.alcampo.es/compra-online/bebidas/licores/c/W1155?q=%3Arelevance&page=",
    # "Vino rosados, frizzantes, dulces y olorosos":"https://www.alcampo.es/compra-online/bebidas/vino-rosados-frizzantes-dulces-y-olorosos/c/W1153?q=%3Arelevance&page=",
    # "Champagne Cavas y Sidras": "https://www.alcampo.es/compra-online/bebidas/champagne-cavas-y-sidras/c/W1156?q=%3Arelevance&page=",
}


dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in bebidas.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                try:
                    dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                except:
                    dict_alcampo["Name"].append("Unkown")

                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)

df_alcampo




Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=2
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=3
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=4
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Bebidas refrescantes https://www.alcampo.es/compra-online/bebidas/bebidas-refrescantes/c/W1103?q=%3Arelevance&page=5
Category: 288 Name: 288 Imag

AttributeError: 'NoneType' object has no attribute 'get_text'